### 1 - Import test data

In [11]:
import yfinance as yf

dataF = yf.download("EURUSD=X", start="2023-4-12", end="2023-6-9", interval='15m')
dataF.iloc[:,:]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-04-12 00:00:00,1.092180,1.092180,1.091941,1.092061,1.092061,0
2023-04-12 00:15:00,1.091822,1.092180,1.091703,1.091822,1.091822,0
2023-04-12 00:30:00,1.091703,1.092061,1.091703,1.091941,1.091941,0
2023-04-12 00:45:00,1.091941,1.092299,1.091941,1.091941,1.091941,0
2023-04-12 01:00:00,1.092419,1.092419,1.091822,1.091941,1.091941,0
...,...,...,...,...,...,...
2023-06-08 22:45:00,1.078632,1.078981,1.078516,1.078632,1.078632,0
2023-06-08 23:00:00,1.078516,1.078865,1.078516,1.078632,1.078632,0
2023-06-08 23:15:00,1.078516,1.078865,1.078516,1.078516,1.078516,0


### 2 - Define your signal function

In [12]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    previous_open = df.Open.iloc[-2]
    previous_close = df.Close.iloc[-2]
    
    # Bearish Pattern
    # selling signal
    if (open>close and 
    previous_open<previous_close and 
    close<previous_open and
    open>=previous_close):
        return 1

    # Bullish Pattern
    # buying signal
    elif (open<close and 
        previous_open>previous_close and 
        close>previous_open and
        open<=previous_close):
        return 2
    
    # No clear pattern
    else:
        return 0

signal = []
signal.append(0)
for i in range(1,len(dataF)):
    df = dataF[i-1:i+1]
    signal.append(signal_generator(df))
dataF["signal"] = signal


In [13]:
dataF.signal.value_counts()

0    3678
2     167
1     139
Name: signal, dtype: int64

In [14]:
x = 0.1
lists_ratio = []
for a in range(1, 20): 
    lists_ratio.append(round(x, 1))
    x += 0.1
a = 1
TP_list = []
for x in range(1, 20): 
    TP_list.append('TP' + str(a))
    a += 1

In [15]:
lists = [[] for i in range(19)]
def data_analysis(df, ratio, df0):
    previous_High = df.High.iloc[-2]
    previous_Low = df.Low.iloc[-2]
    previous_candleR = previous_High - previous_Low
    open = df.Open.iloc[-1]
    TPBuy = open+previous_candleR*ratio
    TPSell = open-previous_candleR*ratio
    signal = df.signal.iloc[-2]
    SLBuy = open-0.003
    SLSell = open+0.003

    if signal == 1:
        for x in range(len(df0.index)):
            if df0.Open.iloc[x] >= SLSell:
                return 'SL'
            elif df0.Low.iloc[x] <= TPSell:
                return 'TP'

    elif signal == 2:
        for x in range(len(df0.index)):
            if df0.Open.iloc[x] <= SLBuy:
                return 'SL'
            elif df0.High.iloc[x] >= TPBuy:
                return 'TP'


TP = []
for x in lists_ratio:
    ratio = x
    TP.append(None)
    for i in range(1,len(dataF)):
        df = dataF[i-1:i+1]
        df0 = dataF[i+1:]
        TP.append(data_analysis(df, ratio, df0))

In [17]:
def split(list_a, chunk_size):

    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]
chunk_size = 3984
p = list(split(TP, chunk_size))
w = 0
for x in TP_list:
    dataF[x] = p[w]
    w += 1

dataF.iloc[:,:]
# dataF.to_excel('data excel.xlsx')
dataF


,Open,High,Low,Close,Adj Close,Volume,signal,TP1,TP2,TP3,...,TP10,TP11,TP12,TP13,TP14,TP15,TP16,TP17,TP18,TP19
Datetime,,,,,,,,,,,,,,,,,,,,,
2023-04-12 00:00:00,1.092180,1.092180,1.091941,1.092061,1.092061,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-04-12 00:15:00,1.091822,1.092180,1.091703,1.091822,1.091822,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-04-12 00:30:00,1.091703,1.092061,1.091703,1.091941,1.091941,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-04-12 00:45:00,1.091941,1.092299,1.091941,1.091941,1.091941,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-04-12 01:00:00,1.092419,1.092419,1.091822,1.091941,1.091941,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-08 22:45:00,1.078632,1.078981,1.078516,1.078632,1.078632,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-06-08 23:00:00,1.078516,1.078865,1.078516,1.078632,1.078632,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2023-06-08 23:15:00,1.078516,1.078865,1.078516,1.078516,1.078516,0,0,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [18]:
import pandas as pd
pieces = []
for col in dataF.iloc[:, 6:27]:
    tmp_series = dataF[col].value_counts()
    tmp_series.name = col
    pieces.append(tmp_series)
df_value_counts = pd.concat(pieces, axis=1)
df_value_counts

,signal,TP1,TP2,TP3,TP4,TP5,TP6,TP7,TP8,TP9,TP10,TP11,TP12,TP13,TP14,TP15,TP16,TP17,TP18,TP19
0,3678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,167.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,139.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TP,NaN,292.0,286.0,283.0,281.0,277.0,269.0,265.0,261.0,255.0,253.0,248.0,244.0,236.0,233.0,231.0,229.0,228.0,227.0,220.0
SL,NaN,14.0,20.0,23.0,25.0,29.0,37.0,41.0,45.0,51.0,53.0,58.0,62.0,70.0,73.0,75.0,77.0,78.0,79.0,85.0


In [20]:
x = dataF[(dataF['Close'] == dataF['Low']) & (dataF['Open'] == dataF['High']) & (dataF['signal'] == 1)]
y = dataF[(dataF['Close'] == dataF['High']) & (dataF['Open'] == dataF['Low']) & (dataF['signal'] == 2)]

In [21]:
y.signal.value_counts()

2    67
Name: signal, dtype: int64

In [22]:
x.signal.value_counts()

1    49
Name: signal, dtype: int64